In [1]:
from datetime import datetime
import uuid
import psycopg2
from rich import print

rozsahy_kategorii = {
    "N": (1.00, 1.17, "Neovlivněné území", "#1a9850"),
    "P": (1.17, 1.22, "Velmi nízká náchylnost k degradaci", "#66bd63"),
    "F1": (1.22, 1.26, "Nízká náchylnost k degradaci", "#a6d96a"),
    "F2": (1.26, 1.32, "Spíše nízká náchylnost k degradaci", "#d9ef8b"),
    "F3": (1.32, 1.37, "Střední náchylnost k degradaci", "#fee08b"),
    "C1": (1.37, 1.41, "Vysoká náchylnost k degradaci", "#fdae61"),
    "C2": (1.41, 1.53, "Velmi vysoká náchylnost k degradaci", "#f46d43"),
    "C3": (1.53, 2.00, "Extrémně vysoká náchylnost k degradaci", "#d73027"),
    "N/A": (None, None, "N/A", "#000000")  # Pro 0 vody
}

def urci_kategorii(vazeny_prumer):
    if vazeny_prumer is None:
        return "N/A", "N/A"

    for kod, (minimum, maximum, nazev, _) in rozsahy_kategorii.items():
        if minimum is not None and maximum is not None:
            if minimum <= vazeny_prumer <= maximum:
                return f"{nazev} ({kod})", kod
        elif kod == "N/A" and vazeny_prumer is None:
            return "N/A", "N/A"

    return "Neznámá kategorie", None

def urci_barvu_klasicke_meritko(meritko):
    meritko_cislo = int(meritko.replace("1:", "").replace(" ", ""))
    if meritko_cislo <= 10000:
        return "#006400", ""
    elif meritko_cislo <= 50000:
        return "#FFA500", "Měřítko má méně přesné rozlišení."
    else:
        return "#FF0000", "Měřítko velmi hrubé - Data je potřeba interpretovat obezřetně."

def urci_barvu_meritko_na_pixel(meritko):
    meritko_cislo = int(meritko.replace("m/px", "").replace(" ", ""))
    if meritko_cislo <= 20:
        return "#006400", ""
    elif meritko_cislo <= 100:
        return "#FFA500", "Měřítko má méně přesné rozlišení."
    else:
        return "#FF0000", "Měřítko velmi hrubé - Data je potřeba interpretovat obezřetně."

def urci_barvu_rok(rok):
    aktualni_rok = datetime.now().year
    rozdil = aktualni_rok - rok
    if rozdil < 3:
        return "#1a9641"
    elif rozdil < 5:
        return "#fdae61"
    else:
        return "#d7191c"

dbname = 'DP_pokus'
user = 'postgres'
password = 'kapli4ky'
host = 'localhost'
port = '5432'

try:
    connection = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
    cursor = connection.cursor()
    print("Připojení k databázi bylo úspěšné")

    sql_query_verohodnost_esai = "SELECT zkratka_txt, datovy_zdroj, indikator_cz, meritko, last_update, source_dat FROM metadata_dat"
    sql_query_funkce = "SELECT AVG(hb_n) AS hodnota_hb, AVG(ctot_n) AS hodnota_uhlik, AVG(evap_n) AS hodnota_evapotranspirace FROM cernovice_esai_na_ctverec;"
    sql_query_esai_hlavni = """
        SELECT AVG(w_veget) AS "Stav vegetace", AVG(w_clim) AS "Stav klimatu", AVG(w_soil) AS "Stav půdy", AVG(w_mgm) AS "Intenzita lidské činnosti" FROM cernovice_esai_na_ctverec;
        """

    sql_query_esai_podrobne = """
        SELECT AVG(dra_w) AS "Propustnost", AVG(par_w) AS "Matečná hornina", AVG(dep_w) AS "Hloubka půdy", AVG(fra_w) AS "Skeletovitost", AVG(tex_w) AS "Textura", AVG(slo_w) AS "Sklonitost svahu", AVG(den_w) AS "Hustota populace", AVG(grw_w) AS "Populační růst", AVG(int_w) AS "Intenzita využití půdy", AVG(w_rai) AS "Průměrný roční úhrn srážek", AVG(arid_w) AS "Index sucha", AVG(asp_w) AS "Orientace svahu", AVG(dry_w) AS "Odolnost vegetace vůči suchu", AVG(ero_w) AS "Schopnost vegetace bránit erozi", AVG(pla_w) AS "Pokryvnost vegetace" FROM cernovice_esai_na_ctverec;
        """

    sql_query_esai = """
        SELECT AVG(w_esai) AS "Zranitelnost krajiny" FROM cernovice_esai_na_ctverec;
        """

    cursor.execute(sql_query_verohodnost_esai)
    data_verohodnost_esai = cursor.fetchall()

    cursor.execute(sql_query_funkce)
    data_funkce = cursor.fetchall()[0] if cursor.rowcount > 0 else None

    cursor.execute(sql_query_esai)
    data_esai = cursor.fetchall()[0][0] if cursor.rowcount > 0 else None

    cursor.execute(sql_query_esai_hlavni)
    data_esai_hlavni = cursor.fetchall()[0] if cursor.rowcount > 0 else None

    cursor.execute(sql_query_esai_podrobne)
    data_esai_podrobne = cursor.fetchall()[0] if cursor.rowcount > 0 else None

    hlavni_esai_kategorie = sorted(
        [("Stav vegetace", data_esai_hlavni[0]), ("Stav klimatu", data_esai_hlavni[1]), ("Stav půdy", data_esai_hlavni[2]), ("Intenzita lidské činnosti", data_esai_hlavni[3])],
        key=lambda x: x[1], reverse=True
    )

    podrobne_esai = sorted(
        [("Propustnost", data_esai_podrobne[0]), ("Matečná hornina", data_esai_podrobne[1]), ("Hloubka půdy", data_esai_podrobne[2]), ("Skeletovitost", data_esai_podrobne[3]), ("Textura", data_esai_podrobne[4]), ("Sklonitost svahu", data_esai_podrobne[5]), ("Hustota populace", data_esai_podrobne[6]), ("Populační růst", data_esai_podrobne[7]), ("Intenzita využití půdy", data_esai_podrobne[8]), ("Průměrný roční úhrn srážek", data_esai_podrobne[9]), ("Index sucha", data_esai_podrobne[10]), ("Orientace svahu", data_esai_podrobne[11]), ("Odolnost vegetace vůči suchu", data_esai_podrobne[12]), ("Schopnost vegetace bránit erozi", data_esai_podrobne[13]), ("Pokryvnost vegetace", data_esai_podrobne[14])],
        key=lambda x: x[1], reverse=True
    )

    if data_funkce:
        hodnota_hb, hodnota_uhlik, hodnota_evapotranspirace = data_funkce
    else:
        hodnota_uhlik = "---"
        hodnota_evapotranspirace = "---"
        hodnota_hb = "---"

    def generuj_tabulku_hlavni_kategorie(hlavni_esai_kategorie, urci_kategorii, rozsahy_kategorii):
        tabulka_html = "<table><tr><th>Téma</th><th>Hodnota</th><th>Kategorie</th></tr>"
        for tema, hodnota in hlavni_esai_kategorie:
            kategorie, kod_kategorie = urci_kategorii(hodnota)
            barva_textu = rozsahy_kategorii[kod_kategorie][3] if kod_kategorie in rozsahy_kategorii else "#000000"
            tabulka_html += f"<tr><td><strong>{tema}</strong></td><td>{hodnota}</td><td style='color: {barva_textu};'>{kategorie}</td></tr>"
        tabulka_html += "</table>"
        return tabulka_html

    def generuj_tabulku_podrobna_temata(podrobne_esai, urci_kategorii, rozsahy_kategorii):
        tabulka_html = "<table><tr><th>Téma</th><th>Hodnota</th><th>Kategorie</th></tr>"
        for tema, hodnota in podrobne_esai:
            kategorie, kod_kategorie = urci_kategorii(hodnota)
            barva_textu = rozsahy_kategorii[kod_kategorie][3] if kod_kategorie in rozsahy_kategorii else "#000000"
            tabulka_html += f"<tr><td>{tema}</td><td>{hodnota}</td><td style='color: {barva_textu};'>{kategorie}</td></tr>"
        tabulka_html += "</table>"
        return tabulka_html

    tabulka_hlavni = generuj_tabulku_hlavni_kategorie(hlavni_esai_kategorie, urci_kategorii, rozsahy_kategorii)
    tabulka_podrobna = generuj_tabulku_podrobna_temata(podrobne_esai, urci_kategorii, rozsahy_kategorii)

    timestamp = datetime.now().strftime("%d-%m-%Y %H:%M:%S")
    id = uuid.uuid1()

    kategorie_esai, kod_kategorie_esai = urci_kategorii(data_esai)

    html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Zranitelnost krajiny</title>
        <meta charset='UTF-8'>
        <meta name='description' content='HTML stránka georeportu zranitelnosti krajiny'>
        <meta name='keywords' content='zranitelnost krajiny, georeport, ekologie'>
        <meta name='author' content='Adéla Petřeková'>
        <style>
            @page {{
                size: A4;
                margin: 10mm 20mm;
                @bottom-center {{
                    content: "Strana " counter(page);
                    font-family: Arial, sans-serif;
                    text-align: center;
                }}
            }}
            body {{
                font-family: Arial, sans-serif;
                margin: 0;
                padding: 0;
            }}
            h1 {{
                background-color: #4CAF50;
                color: #fff;
                padding: 10px;
            }}
            p {{
                padding: 10px;
            }}
            .footer {{
                position: fixed;
                bottom: 0;
                width: 100%;
                text-align: center;
            }}
            .footer hr {{
                width: 100%;
                border: none;
                border-top: 1px solid #000;
                margin: 0;
            }}
            .footer-content {{
                display: flex;
                justify-content: space-between;
                align-items: center;
                padding: 0 0;
            }}
            .footer img {{
                height: 50px;
            }}
            .page_number {{
                font-family: Arial, sans-serif;
            }}
            .indicator-box {{
                border: 2px solid #4CAF50;
                padding: 10px;
                margin: 10px 0;
            }}
            table {{
                width: 100%;
                border-collapse: collapse;
            }}
            th, td {{
                border: 1px solid #000;
                padding: 8px;
                text-align: left;
            }}
            th {{
                background-color: #f2f2f2;
            }}
        </style>
    </head>
    <body>
        <header>
            <h1>Název reportu: Zranitelnost krajiny</h1>
            <p>Lokalita: XXX (GPS souřadnice) <br>
            Souřadnice: MBR extentu <br>
            Datum a čas generování: {timestamp} <br>
            Unikatní identifikátor: {id}
            </p>
        </header>
        <main>
            <p> Tento report podává zprávu o degradaci krajiny na základě metody ESAI ....</p>

            <h2> Krok I: Náchylnost krajiny k degradaci </h2>
            <p> Ve zvoleném území dosahuje výsledná hodnota náchylnosti k degradaci hodnoty <strong>{data_esai}</strong> a je klasifikována jako <span style="color: {rozsahy_kategorii[kod_kategorie_esai][3]};">{kategorie_esai}</span>.</p>
            {tabulka_hlavni}

            <h3> Podřazená témata </h3>
            {tabulka_podrobna}

    """

    serazene_podrobne_esai = sorted(podrobne_esai, key=lambda x: x[1], reverse=True)
    nejhorsi_indikatory = [f"<em>{tema}</em>" for tema, _ in serazene_podrobne_esai[:3]]
    html += "<div class='indicator-box'> Nejhorší 3 indikátory v zájmovém území jsou: " + ", ".join(nejhorsi_indikatory) + ".</div>"

    html += f"""
            
            <h3> Věrohodnost dat </h3>
            <table>
                <tr>
                    <th> Zkratka </th>
                    <th> Název </th>
                    <th> Datový zdroj </th>
                    <th> Garant </th>
                    <th> Měřítko </th>
                    <th> Rok aktualizace </th>
                </tr>
    """

    for zkratka_txt, datovy_zdroj, indikator_cz, meritko, last_update, source_dat in data_verohodnost_esai:
        if "1:" in meritko:
            barva, poznamka = urci_barvu_klasicke_meritko(meritko)
        elif "m/px" in meritko:
            barva, poznamka = urci_barvu_meritko_na_pixel(meritko)
        barva_rok = urci_barvu_rok(last_update)

        html += f"""
                <tr>
                    <td> {zkratka_txt} </td>
                    <td> {indikator_cz} </td>
                    <td> {datovy_zdroj} </td>
                    <td> {source_dat} </td>
                    <td style="color: {barva};"> {meritko} {poznamka} </td>
                    <td style="color: {barva_rok};"> {last_update} </td>
                </tr>
        """

    html += f"""
            </table>
            <h2> Krok II: Posouzení funkčnosti krajiny </h2>
            <p> slouží k nalezení krajinných segmentů jejichž degradace bude mít velký/malý význam dopad na funkčnost krajiny </p>
            <table>
                <tr>
                    <th> </th>
                    <th> Hodnota </th>
                    <th> Kategorie </th>
                </tr>
                <tr>
                    <td> Zásoba uhlíku ve vegetaci </td>
                    <td> {hodnota_uhlik} </td>
                    <td> Kategorie 1 </td>
                </tr>
                <tr>
                    <td> Evapotranspirace </td>
                    <td> {hodnota_evapotranspirace} </td>
                    <td> Kategorie 1 </td>
                </tr>
                <tr>
                    <td> Ekologická hodnota biotopu </td>
                    <td> {hodnota_hb} </td>
                    <td> Kategorie 1 </td>
                </tr>
                <tr>
                    <td> Vodoretence </td>
                    <td> Hodnota 1 </td>
                    <td> Kategorie 1 </td>
                </tr>
                <tr>
                    <td> Integrovaná hodnota </td>
                    <td> --- </td>
                    <td> Kategorie 1 </td>
                </tr>
            </table>
            <h3> Věrohodnost dat </h3>
            <table>
                <tr>
                    <th> Název </th>
                    <th> Datový zdroj geometrie/KB </th>
                    <th> Garant </th>
                    <th> Měřítko </th>
                    <th> Rok aktualizace </th>
                </tr>
                <tr>
                    <td> Indikátor 1 </td>
                    <td> Hodnota 1 </td>
                    <td> Věrohodnost 1 </td>
                    <td> meritko </td>
                    <td> ROK </td>
                </tr>
            </table>
            <h2> Krok III: Odolnost krajiny </h2>
            <h2> Krok IV: Syntéza/lokalizace prioritních opatření </h2>
            <h2> Krok V: Typizace prioritních opatření </h2>
        </main>
        <footer>
            <div class="footer">
                <hr>
                <div class="footer-content">
                    <img src="katedra_logo.png" alt="Logo KGI">
                    <img src="logo_czechglobe.png" alt="Logo CzechGlobe">
                </div>
            </div>
        </footer>
    </body>
    </html>
    """

    with open("report.html", "w", encoding="utf-8") as file:
        file.write(html)

    print("HTML report byl úspěšně vygenerován.")

except Exception as error:
    print(f"Chyba při připojování k databázi: {error}")

finally:
    if connection:
        cursor.close()
        connection.close()

Připojení k databázi bylo úspěšné

HTML report byl úspěšně vygenerován.

In [2]:
# Vytvoření HTML souboru
with open("index.html", "w", encoding="utf-8") as f:
    f.write(html)

print("HTML report byl úspěšně vygenerován.")

# (Pokud chcete otevřít soubor v prohlížeči, odkomentujte následující řádky)
# import webbrowser
# webbrowser.open("index.html")

HTML report byl úspěšně vygenerován.

In [3]:
# output html as a PDF
# first, import the necessary modules

from weasyprint import HTML
# then, convert the HTML to a PDF
HTML("index.html").write_pdf("index.pdf")

print("Generated index.pdf")
#otevření pdf souboru
import webbrowser
webbrowser.open("index.pdf")

Generated index.pdf

True